Naujas kodas alfų peakams ištraukti

In [1]:
import numpy as np
import pandas as pd
import scipy.io as spio
from openpyxl import Workbook

# load Matlab matrixes, extract parameters
# update the paths to the files accordingly
iaf_mat = spio.loadmat('')['iaf']
power_mat = spio.loadmat('')['broadpow']
subj_id_mat = spio.loadmat('')['ID_all'][0]
channels_mat = spio.loadmat('')['chan_label'][0]

names = []
for x in range(len(subj_id_mat)):
    name = subj_id_mat[x][0]
    names.append(name)

channels = []
for y in range(len(channels_mat)):
    channel = channels_mat[y][0]
    channels.append(channel)

iaf = np.array(iaf_mat)
power = np.array(power_mat)

iaf_ch_id = pd.DataFrame(iaf, columns=channels, index=[int(''.join(filter(str.isdigit, name))) for name in names])
power_ch_id = pd.DataFrame(power, columns=channels, index=[int(''.join(filter(str.isdigit, name))) for name in names])

# Reset the index and rename the column
iaf_ch_id = iaf_ch_id.reset_index().rename(columns={'index': 'id'})
power_ch_id = power_ch_id.reset_index().rename(columns={'index': 'id'})


# Create a mapping dictionary for the 'group' column
group_mapping = {
    (0, 100): 'OC',
    (100, 200): 'IUD',
    (200, 300): 'NCF',
    (300, 400): 'NCL',
    (400, float('inf')): 'M',
}

# Add the 'group' column 
iaf_ch_id.insert(1, 'group', pd.cut(iaf_ch_id['id'], bins=[0, 100, 200, 300, 400, float('inf')], labels=['OC', 'IUD', 'NCF', 'NCL', 'M']))
power_ch_id.insert(1, 'group', pd.cut(power_ch_id['id'], bins=[0, 100, 200, 300, 400, float('inf')], labels=['OC', 'IUD', 'NCF', 'NCL', 'M']))

nan_threshold = 31

# Initialize empty lists for filtered data
filtered_data_iaf = []
filtered_data_pw = []
subjects_exceeding_nan = []

for (iaf_row, pw_row) in zip(iaf_ch_id.iterrows(), power_ch_id.iterrows()):
    iaf_index, iaf_data = iaf_row
    pw_index, pw_data = pw_row
    
    # Assuming 'id' is the same for both iaf and power datasets
    subject_id = iaf_data['id']

    # Count NaNs (using either iaf or power, as NaN count is the same)
    nan_count = iaf_data.isnull().sum()

    # Filter both iaf and power data based on the NaN count
    if nan_count <= nan_threshold:
        filtered_data_iaf.append(iaf_data)
        filtered_data_pw.append(pw_data)
    else:
        subjects_exceeding_nan.append((subject_id, nan_count))

# Convert lists to dataframes
filtered_iaf = pd.DataFrame(filtered_data_iaf)
filtered_power = pd.DataFrame(filtered_data_pw)

# Print subjects with more than 31 NaN values
print("Subjects with more than 31 NaN values:")
for subject, nan_count in subjects_exceeding_nan:
    print(f"Subject ID: {subject}, NaN count: {nan_count}")



FileNotFoundError: [Errno 2] No such file or directory: '.mat'

In [5]:
import scipy.io as sio
import pandas as pd
import numpy as np

# Load the MATLAB alpha_params data
mat_file_path = r'E:\Analysis\Analysis\params.mat'
mat_data = sio.loadmat(mat_file_path)

# Extract alpha parameters from the .mat file
alpha_params = mat_data['alpha_params'].squeeze()

# Create lists to store data
subject_ids = []
alpha_powers = []
iaf_freqs = []

# Iterate over subjects and extract data
for subject in alpha_params:
    subject_id = subject['ID'][0]
    subject_ids.append(subject_id)
    
    # Get alpha power and frequency data for all channels
    alpha_powers.append(subject['power'].squeeze())
    iaf_freqs.append(subject['freq'].squeeze())

# Convert to DataFrames for easier processing
power_df = pd.DataFrame(alpha_powers, index=subject_ids)
freq_df = pd.DataFrame(iaf_freqs, index=subject_ids)

# Set NaN threshold
nan_threshold = 31

# Filter data based on NaN count
filtered_power = power_df[power_df.isnull().sum(axis=1) <= nan_threshold]
filtered_freq = freq_df[freq_df.isnull().sum(axis=1) <= nan_threshold]

# Identify subjects exceeding the NaN threshold
subjects_exceeding_nan = power_df.index[power_df.isnull().sum(axis=1) > nan_threshold].tolist()

# Output the results
print("Subjects with more than 31 NaN values:")
for subject_id in subjects_exceeding_nan:
    nan_count = power_df.loc[subject_id].isnull().sum()
    print(f"Subject ID: {subject_id}, NaN count: {nan_count}")

# Optionally save the filtered data to new files
filtered_power.to_csv('filtered_power.csv')
filtered_freq.to_csv('filtered_freq.csv')


Subjects with more than 31 NaN values:
Subject ID: OC_13_R2_ch256_flt1-45_flt50_rejectedarte_rch_imp_ICA_prep_rf_int, NaN count: 47
Subject ID: OC_20_R2_ch256_flt1-45_flt50_rejectedarte_rch_imp_ICA_prep_rf_int, NaN count: 43
Subject ID: OC_32_R2_ch256_flt1-45_flt50_rejectedarte_rch_imp_ICA_prep_rf_int, NaN count: 56


In [8]:
# Extracting Data From the Specific Channel(s)
# The folowing code is to:

# Extract the data from specific channel
# Calculate median parameter values for each group
# Save the data to Excel

# Specify the channel
channel = '61'

# power and iaf from a specified channel(s) for each subject
power_pz = filtered_power[['id', 61]]
iaf_pz = filtered_iaf[['id', 61]]

data_frames = [
    ('iaf_Pz', iaf_pz),
    ('power_pz', power_pz),
    ('iaf_unfiltered_all_chans', iaf_ch_id),
    ('power_unfiltered_all_chans', power_ch_id),
    ('iaf_each_ch_sbj_removed_subj', filtered_iaf),
    ('power_each_ch_sbj_removed_subj', filtered_power),
]

with pd.ExcelWriter('iaf_power.xlsx') as writer:
    for sheet_name, data_frame in data_frames:
        data_frame.to_excel(writer, sheet_name=sheet_name, index=False)

KeyError: "['id'] not in index"

In [7]:
# Verify columns in filtered_power and filtered_iaf
print(filtered_power.columns)
print(filtered_iaf.columns)

# Assuming the channel name 'Pz' is correct
# Extracting the data
channel = 'Pz'

# Ensure these DataFrames are correctly initialized and columns are present
power_pz = filtered_power[['id', 'group', channel]]
iaf_pz = filtered_iaf[['id', 'group', channel]]

# Ensure iaf_ch_id and power_ch_id are correctly initialized (if used)
# Example:
# iaf_ch_id = filtered_iaf[['id', 'group', 'other_channels']]
# power_ch_id = filtered_power[['id', 'group', 'other_channels']]

data_frames = [
    ('iaf_Pz', iaf_pz),
    ('power_pz', power_pz),
    ('iaf_unfiltered_all_chans', iaf_ch_id),
    ('power_unfiltered_all_chans', power_ch_id),
    ('iaf_each_ch_sbj_removed_subj', filtered_iaf),
    ('power_each_ch_sbj_removed_subj', filtered_power),
]

# Writing the data to an Excel file
with pd.ExcelWriter('iaf_power.xlsx') as writer:
    for sheet_name, data_frame in data_frames:
        data_frame.to_excel(writer, sheet_name=sheet_name, index=False)


RangeIndex(start=0, stop=62, step=1)


NameError: name 'filtered_iaf' is not defined